In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,872 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126102 files and dire

In [1]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [3]:
# ▶ 필요한 패키지 import
import io
import pandas as pd
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.arima.model import ARIMA

# --- 1. 데이터 로드 및 전처리 ---
excel_path = '/content/시군구_합계출산율__모의_연령별_출산율_20250517112929_분석(전년_대비_증감,증감률).xlsx'

# ▶ 엑셀 데이터 불러오기 (멀티헤더 처리)
df_raw = pd.read_excel(excel_path, sheet_name='데이터', header=[0, 1])
df_raw.columns = df_raw.columns.map(lambda x: x[0] if '시군구별' in x else x)

# ▶ 합계출산율 원데이터 컬럼 추출 (2015~2023)
target_cols = [
    col for col in df_raw.columns
    if isinstance(col, tuple)
    and col[1] == '합계출산율'
    and col[0].isdigit()
    and 2015 <= int(col[0]) <= 2023
]

# ▶ 대상 지역
regions = [
    "전국",
    "서울특별시","부산광역시","대구광역시","인천광역시",
    "광주광역시","대전광역시","울산광역시","세종특별자치시",
    "경기도","강원특별자치도","충청북도","충청남도",
    "전라북도","전라남도","경상북도","경상남도","제주특별자치도"
]

# ▶ 2015~2024년 시계열 DataFrame 생성
df_all = pd.DataFrame(index=range(2015, 2025), columns=regions, dtype=float)

# ▶ 2015~2023 실제 데이터 채우기
for region in regions:
    row = df_raw[df_raw['시군구별'] == region][target_cols]
    row.columns = [int(c[0]) for c in target_cols]
    ts = row.T.iloc[:, 0]
    df_all.loc[ts.index, region] = pd.to_numeric(ts.values, errors='coerce')

# ▶ 2024년 예측 (각 지역별 Holt vs ARIMA)
for region in regions:
    train = df_all.loc[2015:2022, region]
    actual23 = df_all.loc[2023, region]

    # Holt 선형 추세
    holt = Holt(train).fit()
    pred23_h = holt.forecast(1).iloc[0]
    pred24_h = holt.forecast(2).iloc[1]

    # ARIMA(1,1,0)
    arima = ARIMA(train, order=(1,1,0)).fit()
    pred23_a = arima.forecast(1).iloc[0]
    pred24_a = arima.forecast(2).iloc[1]

    # 2023 예측 정확도 비교 → 더 나은 모델로 2024 예측값 선택
    err_h = abs(pred23_h - actual23)
    err_a = abs(pred23_a - actual23)

    df_all.loc[2024, region] = pred24_h if err_h < err_a else pred24_a

# ▶ 결과 저장 및 출력
df_all.index.name = '연도'
df_all.to_csv('시도별_합계출산율_2015_2024_예측포함.csv', encoding='utf-8-sig')
print(df_all)


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         전국     서울특별시  부산광역시     대구광역시     인천광역시     광주광역시     대전광역시  울산광역시  \
연도                                                                            
2015  1.239  1.001000  1.139  1.216000  1.216000  1.207000  1.277000  1.486   
2016  1.172  0.940000  1.095  1.186000  1.144000  1.168000  1.192000  1.418   
2017  1.052  0.836000  0.976  1.067000  1.007000  1.053000  1.075000  1.261   
2018  0.977  0.761000  0.899  0.987000  1.006000  0.972000  0.952000  1.131   
2019  0.918  0.717000  0.827  0.932000  0.940000  0.913000  0.883000  1.084   
2020  0.837  0.642000  0.747  0.807000  0.829000  0.811000  0.805000  0.984   
2021  0.808  0.626000  0.728  0.785000  0.778000  0.896000  0.810000  0.940   
2022  0.778  0.593000  0.723  0.757000  0.747000  0.844000  0.842000  0.848   
2023  0.721  0.552000  0.664  0.702000  0.694000  0.706000  0.787000  0.814   
2024  0.644  0.545207  0.635  0.655839  0.602467  0.764968  0.894801  0.712   

       세종특별자치시       경기도   강원특별자치도   충청북도      충청남도

In [5]:
import pandas as pd
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.arima.model import ARIMA

# 1. 엑셀 파일 로드
excel_path = '/content/시군구_합계출산율__모의_연령별_출산율_20250517112929_분석(전년_대비_증감,증감률).xlsx'  # Colab이면 업로드된 파일명
df_raw = pd.read_excel(excel_path, sheet_name='데이터', header=[0, 1])
df_raw.columns = df_raw.columns.map(lambda x: x[0] if '시군구별' in x else x)

# 2. 합계출산율 원데이터 컬럼 추출
target_cols = [
    col for col in df_raw.columns
    if isinstance(col, tuple)
    and col[1] == '합계출산율'
    and col[0].isdigit()
    and 2015 <= int(col[0]) <= 2023
]

# 3. 지역 목록
regions = [
    "전국", "서울특별시", "부산광역시", "대구광역시", "인천광역시",
    "광주광역시", "대전광역시", "울산광역시", "세종특별자치시",
    "경기도", "강원특별자치도", "충청북도", "충청남도",
    "전라북도", "전라남도", "경상북도", "경상남도", "제주특별자치도"
]

# 4. 전체 시계열 데이터프레임
df_all = pd.DataFrame(index=range(2015, 2025), columns=regions, dtype=float)
summary = []

# 5. 데이터 구성 (2015~2023)
for region in regions:
    row = df_raw[df_raw['시군구별'] == region][target_cols]
    row.columns = [int(c[0]) for c in target_cols]
    ts = row.T.iloc[:, 0]
    df_all.loc[ts.index, region] = pd.to_numeric(ts.values, errors='coerce')

# 6. 모델 성능 평가 및 2024 예측
for region in regions:
    train = df_all.loc[2015:2022, region]
    actual_2023 = df_all.loc[2023, region]

    # Holt
    holt_model = Holt(train).fit()
    pred_2023_h = holt_model.forecast(1).iloc[0]
    pred_2024_h = holt_model.forecast(2).iloc[1]

    # ARIMA
    arima_model = ARIMA(train, order=(1, 1, 0)).fit()
    pred_2023_a = arima_model.forecast(1).iloc[0]
    pred_2024_a = arima_model.forecast(2).iloc[1]

    # RMSE
    rmse_h = abs(pred_2023_h - actual_2023)
    rmse_a = abs(pred_2023_a - actual_2023)

    best_model = 'Holt' if rmse_h < rmse_a else 'ARIMA'
    best_pred = pred_2024_h if best_model == 'Holt' else pred_2024_a
    df_all.loc[2024, region] = best_pred

    summary.append({
        '지역': region,
        'Holt RMSE': round(rmse_h, 4),
        'ARIMA RMSE': round(rmse_a, 4),
        '선택된 모델': best_model
    })

# 7. 결과 출력
df_summary = pd.DataFrame(summary)
print(df_summary)



/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         지역  Holt RMSE  ARIMA RMSE 선택된 모델
0        전국     0.0100      0.0320   Holt
1     서울특별시     0.0200      0.0145  ARIMA
2     부산광역시     0.0150      0.0552   Holt
3     대구광역시     0.0044      0.0377   Holt
4     인천광역시     0.0287      0.0356   Holt
5     광주광역시     0.0980      0.1253   Holt
6     대전광역시     0.0814      0.0817   Holt
7     울산광역시     0.0340      0.0378   Holt
8   세종특별자치시     0.0622      0.0617  ARIMA
9       경기도     0.0177      0.0610   Holt
10  강원특별자치도     0.0249      0.0678   Holt
11     충청북도     0.0710      0.0752   Holt
12     충청남도     0.0159      0.0155  ARIMA
13     전라북도     0.0290      0.0049  ARIMA
14     전라남도     0.0785      0.0373  ARIMA
15     경상북도     0.0020      0.0397   Holt
16     경상남도     0.0400      0.0188  ARIMA
17  제주특별자치도     0.0253      0.0669   Holt
